Install tesseract

In [ ]:
!sudo apt install tesseract-ocr
!pip install pytesseract

install selenium
and
chrome driver (also define path)

In [ ]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

import dependancies

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from PIL import Image, ImageCms, ImageOps, ImageFilter
import numpy as np
import pytesseract
import shutil
import cv2
import time

helper functions

In [ ]:
def step1():
	# open webpage
	driver.get(URL)

	# save img
	imdata = driver.find_element(By.ID,"imgCaptcha")
	with open(path, 'wb') as file:
	    file.write(imdata.screenshot_as_png)

def step2():
	# convert to inverted mask and save
	im = cv2.imread(path)
	gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
	thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
	horizontal_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (25, 1))
	Mask = cv2.morphologyEx(thresh, cv2.MORPH_OPEN,horizontal_kernel, iterations=2)
	#Mask = cv2.bitwise_not(Mask)
	cv2.imwrite("old.png", Mask)

	# reinvert mask
	img = Image.open("old.png")
	img = img.convert("RGBA")
	datas = img.getdata()
	newData = []
	for item in datas:
	    if item[0] == 0 and item[1] == 0 and item[2] == 0:
	        newData.append((255, 255, 255, 0))
	    else:
	        newData.append(item)
	img.putdata(newData)

	# paste mask on img and save new img
	background = Image.open(path)
	background = background.convert("RGBA")
	background.paste(img,mask=img)
	background.save("new.png","PNG")

def step3(im): # solve captcha
	im = Image.open(im)
	im = im.crop((5,5,115,35))
	rgb = ImageCms.createProfile(colorSpace='sRGB')
	lab = ImageCms.createProfile(colorSpace='LAB')
	transform = ImageCms.buildTransform(inputProfile=rgb, outputProfile=lab, inMode='RGB', outMode='LAB')
	lab_im = ImageCms.applyTransform(im=im, transform=transform)
	l, a, b = lab_im.split()
	im=l
	im = im.filter(ImageFilter.MinFilter(3))
	result = pytesseract.image_to_string(im)
	l=[]
	l.append(result.strip())
	if l[0]==" " or l[0]=="":
		step1()
		step2()
		l[0]=step3("new.png")
	return l[0]

def step4(enroll,fname): # print webpage
	enr = driver.find_element(By.ID,"txtEnrollNo")
	captex = driver.find_element(By.ID,"CodeNumberTextBox")
	enr.send_keys(enroll)
	captex.send_keys(ans)
	captex.send_keys(Keys.RETURN)
	S = lambda X: driver.execute_script('return document.body.parentNode.scroll'+X)
	driver.set_window_size(S('Width'),S('Height'))                                                                                                              
	driver.find_element(By.TAG_NAME,"body").screenshot(fname)
	#driver.quit()
	#time.sleep(5)

main function

In [ ]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument("--incognito")

ser = Service("chromedriver")
driver = webdriver.Chrome(service=ser,options=chrome_options)

URL = "https://www.students.gtu.ac.in/"
path="cap.jpg"

for i in range(31,33+1): # 31 to 33
	enroll = "1902801110{}".format(i)
	fname = "./outputs/1902801110{}.png".format(i)
	step1()
	step2()
	ans=step3("new.png")
	step4(enroll,fname)